# Clothes Classifier

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

import pathlib
import glob
import os
import re

from PIL import Image
from pathlib import Path
from datetime import datetime

2021-11-22 10:19:50.112375: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-22 10:19:50.112394: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter("model_fmnist_edge.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details() #28x28x3
output_details = interpreter.get_output_details()

[{'name': 'serving_default_conv2d_input:0', 'index': 0, 'shape': array([ 1, 28, 28,  1], dtype=int32), 'shape_signature': array([-1, 28, 28,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [ ]:
def load_labels(path):
    """Loads the labels file. Supports files with or without index numbers."""
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        labels = []
        count=0
        
        #for row_number, content in enumerate(lines):
        for line in lines:
            labels.append(line)
    return labels


In [8]:
def load_labels(path):
    """Loads the labels file. Supports files with or without index numbers."""
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        labels = []
        count=0
        
        #for row_number, content in enumerate(lines):
        for line in lines:
            labels.append(line)
    return labels


T-shirt_top



In [30]:
# Load the labels
labels = load_labels('labels.txt')

###### Prepare the image or the imageset.
folder = "test_images"
files = glob.glob(folder+'/*.jpg')

#NxHxWxC, H:1, W:2
height=input_details[0]['shape'][1]
width=input_details[0]['shape'][2]

#Results:
hit=0
miss=0
accumTime=0

for file in files:
    print(file)
    img = Image.open(file).resize((height, width))
    # add N dim
    input_data = np.expand_dims(img, axis=0)
    input_data = input_data[..., np.newaxis]
    #Input data must be float
    input_data = (2.0 / 255.0) * np.float32(input_data) - 1.0
    print(input_data.shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    startTime = datetime.now()
    interpreter.invoke()
    delta = datetime.now() - startTime
    accumTime=accumTime+(delta.total_seconds() * 1000)

    output_data = interpreter.get_tensor(output_details[0]['index'])
    #print(output_data)
    
    myPath=os.path.splitext(file)[0]
    filename=Path(myPath).stem
    #res = re.findall('([a-zA-Z ]*)\d*.*', filename)
    res=filename.split('_')[0]
    print(res)
    predicted_label = np.argmax(output_data)
    prediction=(labels[predicted_label])
    prediction=prediction.rstrip().lower()
    print(prediction)

    if(res==prediction):
        hit+=1
    else:
        miss+=1

    print("Image is " +res+". Prediction is "+ prediction+".")
            
print("Hits="+str(hit))
print("Misses="+str(miss))
print("Average time:", '%.1f' % (accumTime/46), "ms\n")


test_images/shirt_5.jpg
(1, 28, 28, 1)
shirt
shirt
Image is shirt. Prediction is shirt.
test_images/t-shirt_top_18.jpg
(1, 28, 28, 1)
t-shirt
shirt
Image is t-shirt. Prediction is shirt.
test_images/coat_48.jpg
(1, 28, 28, 1)
coat
coat
Image is coat. Prediction is coat.
test_images/dress_6.jpg
(1, 28, 28, 1)
dress
shirt
Image is dress. Prediction is shirt.
test_images/t-shirt_top_21.jpg
(1, 28, 28, 1)
t-shirt
trouser
Image is t-shirt. Prediction is trouser.
test_images/trouser_22.jpg
(1, 28, 28, 1)
trouser
trouser
Image is trouser. Prediction is trouser.
test_images/coat_46.jpg
(1, 28, 28, 1)
coat
coat
Image is coat. Prediction is coat.
test_images/sneaker_26.jpg
(1, 28, 28, 1)
sneaker
sandal
Image is sneaker. Prediction is sandal.
test_images/bag_34.jpg
(1, 28, 28, 1)
bag
bag
Image is bag. Prediction is bag.
test_images/dress_45.jpg
(1, 28, 28, 1)
dress
bag
Image is dress. Prediction is bag.
test_images/t-shirt_top_15.jpg
(1, 28, 28, 1)
t-shirt
pullover
Image is t-shirt. Prediction is